In [ ]:
# Initialize trace environment and set up some helper methods
from helpers import trace_env
from helpers import image_formats
from helpers import jupyter_helpers
from helpers import replay
from helpers import trace as trace_info
from helpers import vulkan
import matplotlib
import os
import ipywidgets
import numpy as np

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [32, 18]

from importlib import reload

def reload_helpers():
    reload(trace_env)
    reload(image_formats)
    reload(jupyter_helpers)
    reload(replay)
    reload(trace_info)
    reload(vulkan)


reload_helpers()

pathdir = globals()['_dh'][0]
env = trace_env.trace_env(os.path.normpath(os.path.join(pathdir, "..", "build", "RelWithDebInfo")))
env.ignore_debug_messages = True
env.ignore_layer_info_messages = True


In [ ]:
# Select and load trace file
from ipyfilechooser import FileChooser

reload_helpers()

fc = FileChooser()
if env.current_trace:
    fc.default_path = os.path.dirname(env.current_trace)
    fc.default_filename = os.path.basename(env.current_trace)

fc.filter_pattern = "*.trace"
fc.title = "<b>Select Trace</b>"
trace = None
def change_file(chooser):
    global trace
    current_trace = os.path.join(chooser.selected_path, chooser.selected_filename)
    trace =  trace_info.load_trace(env, current_trace)
fc.register_callback(change_file)
display(fc)

In [ ]:
# Report stats about the trace
import json
reload_helpers()

cmd_stats = trace.get_stats()
print(json.dumps(cmd_stats, indent="    "))


In [ ]:
# Count the draw commands actually submitted
from collections import defaultdict

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & trace_info.MID_EXECUTION) == 0)]

submitted_draws = []
submitted_dispatches = []
for x in queue_submits:
    executed_cbs = trace.executed_commands[x]
    for y in executed_cbs:
        submitted_draws.extend([x for x in y[1] if "Draw" in trace.commands[x]["name"]])
        submitted_dispatches.extend([x for x in y[1] if "Dispatch" in trace.commands[x]["name"]])

l = defaultdict(list)
for x in submitted_draws:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Draws: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

l = defaultdict(list)
for x in submitted_dispatches:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Dispatches: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

In [ ]:
# Render trace on the screen
reload_helpers()

create_swapchain = [x for x in trace.commands if x["name"] == "vkCreateSwapchainKHR"][0]

opts = replay.replay_options(env, trace)
opts.dont_use_callback_swapchain()
opts.add_layer(os.path.abspath("on_screen.cpp"), {
    "start_idx": 1,
    "width": create_swapchain["pCreateInfo"]["imageExtent"]["width"],
    "height": create_swapchain["pCreateInfo"]["imageExtent"]["height"],
}, None)

replay.replay(env, opts)

In [ ]:
# Collect screenshots from the trace (Start with the first 10 images that were properly rendered)
screenshots_to_collect = 10
reload_helpers()

import itertools
import base64
from matplotlib import pyplot

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & trace_info.MID_EXECUTION) == 0)]

idx = 0
executed_cbs = trace.executed_commands[queue_submits[idx]]
draws = []
while (len(draws) == 0):
    idx += 1
    executed_cbs = trace.executed_commands[queue_submits[idx]]

    for y in range(len(executed_cbs)):
        draws.extend([(y, x) for x in range(len(executed_cbs[y][1])) if "Draw" in trace.commands[executed_cbs[y][1][x]]["name"]])

draws = itertools.groupby(draws, lambda x: x[0])
total_draws = 0
config = {"screenshot_locations": [
    {
        "submit_index": queue_submits[idx],
        "command_buffer_indices": []
    }
], "num_images_per_draw": 1}

for x in draws:
    cbs = {
        "command_buffer_index": x[0],
    }
    draw_calls = list(x[1])
    draws_to_take = screenshots_to_collect - total_draws
    if len(draw_calls) < draws_to_take:
        draws_to_take = len(draw_calls)
    cbs["indices"] = [x[1] for x in draw_calls[:draws_to_take]]
    config["screenshot_locations"][0]["command_buffer_indices"].append(cbs)
opts = replay.replay_options(env, trace)

def on_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'[{timestamp}] {level} :: {message}')

opts.set_message_callback(on_message)

def on_data(timestamp, data):
    img = image_formats.ToNumpyArray(data["format"], data["width"], data["height"], base64.b64decode(data["data"]))
    # Todo: Actually display this data!
    pyplot.imshow(img)
    pyplot.show()

def on_layer_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'screenshot.cpp:: [{timestamp}] {level} :: {message}')

opts.add_layer(os.path.abspath("screenshot.cpp"), config, data_callback=on_data, message_callback=on_layer_message)

replay.replay(env, opts)

In [ ]:
# Collect screenshots from something rendering color!
screenshots_to_collect = 1000
reload_helpers()
import itertools
import base64
import re
import sys
import PIL
from matplotlib import pyplot
from IPython.display import display, HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128


# Step 1 find all of the 
queue_submits = trace.find_command_indices("vkQueueSubmit")
renderpass_creates = {trace.commands[x]["pRenderPass"]: x for x in trace.find_command_indices("vkCreateRenderPass", True)}

def get_first_renderpass_with_color():
    for x in queue_submits:
        rp_begins = trace.get_submitted_commands_matching(x, re.compile("vkCmdBeginRenderPass"))
        for c in rp_begins:
            for d in c[1]:
                rp_begin = trace.commands[d[1]]
                render_pass = rp_begin["pRenderPassBegin"]["renderPass"]
                rp_creation = trace.commands[renderpass_creates[render_pass]]
                color_attachments = [x for x in rp_creation["pCreateInfo"]["pSubpasses"][0]["pColorAttachments"] if x["attachment"] != vulkan.VK_ATTACHMENT_UNUSED]
                if (len(color_attachments)):
                    has_draws = trace.get_submitted_commands_matching(x, re.compile(".*Draw.*"))
                    if not has_draws:
                        continue
                    return (x, c[0], d[0])
    return None

cb_with_color = get_first_renderpass_with_color()
if cb_with_color == None:
    print("No renderpasses that actally draw anything (odd)")
    sys.exit(-1)

draws_in_submit = trace.get_submitted_commands_matching(cb_with_color[0], re.compile(".*Draw.*"))
cbs_matching = [x for x in range(len(draws_in_submit)) if draws_in_submit[x][0] == cb_with_color[1]]
draws = [x[0] for x in draws_in_submit[cbs_matching[0]][1] if x[0] > cb_with_color[2]][:screenshots_to_collect]
config = {"screenshot_locations": [
    {
        "submit_index": cb_with_color[0],
        "command_buffer_indices": [{
            "command_buffer_index": cbs_matching[0],
            "indices": draws
        },]
    }
], "num_images_per_draw": 1}

opts = replay.replay_options(env, trace)

def on_message(timestamp, level, message):
    return

opts.set_message_callback(on_message)
imgs = []
def on_data(timestamp, data):
    if (type(data) == str):
        print(data)
        return
    global imgs
    dat = base64.b64decode(data["data"])
    img = image_formats.ToNumpyArray(data["format"], data["width"], data["height"], dat)
    imgs.append(img.astype('uint8'))

def on_layer_message(timestamp, level, message):
    if level == "Debug":
        return
    if level == "Info":
        return
    print(f'screenshot.cpp:: [{timestamp}] {level} :: {message}')

opts.add_layer(os.path.abspath("screenshot.cpp"), config, data_callback=on_data, message_callback=on_layer_message)

display(ipywidgets.widgets.Label(f"Getting images for {len(draws)} draw calls"))
replay.replay(env, opts)
#fig, anim = jupyter_helpers.plot_sequence_images(imgs[1:])
#pyplot.show(fig)


In [ ]:
def show(i):
    pyplot.imshow(np.flip(imgs[i], 0))
ipywidgets.interact(show, i=ipywidgets.IntSlider(min=0, max=len(imgs) - 1, continuous=False))